In [1]:
import fitsio
from fitsio import FITS, FITSHDR
from astropy import wcs
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
git_path = os.environ['GIT_REPO_LOC']
data_path = os.path.join(git_path, 'DECamSurvey.jl/dat')
image_filename = os.path.join(data_path, 'c4d_160302_094418_oki_z_v1.fits')
mask_filename = os.path.join(data_path, 'c4d_160302_094418_ood_z_v1.fits')
dr3_object = fitsio.read_header(image_filename)['OBJECT']

In [10]:
# Load an image.
image_index = 3
image = fitsio.read(image_filename, ext=image_index)
image_mask = fitsio.read(mask_filename, ext=image_index)
header = fitsio.read_header(image_filename, ext=image_index)
im_wcs = wcs.WCS(header)

print 'Good pixel proportion:'
print np.sum(image_mask == 0) / float(np.prod(image_mask.shape))
image[image_mask > 0] = float('nan')
# plt.matshow(np.log(image), interpolation="nearest")

Good pixel proportion:
0.974422192838


In [46]:
# Load the catalog.

# First select the bricks that overlap with the image.
ra_corners = [ header['COR%dRA1' % (i + 1)] for i in range(4)]
dec_corners = [ header['COR%dDEC1' % (i + 1)] for i in range(4)]
brick_select = \
  'ra2 > {ramin} && ra1 < {ramax} && dec2 > {decmin} && dec1 < {decmax}'.format(
      ramin=np.min(ra_corners), ramax=np.max(ra_corners),
      decmin=np.min(dec_corners), decmax=np.max(dec_corners))

brick_filename = os.path.join(data_path, 'survey-bricks.fits')
brick_fits = fitsio.FITS(brick_filename)
keep_rows = brick_fits[1].where(brick_select) 
bricks = brick_fits[1]['brickname'][keep_rows]
print bricks



['1984p110' '1986p110' '1984p112' '1987p112']


In [52]:
tractor_filename = os.path.join(data_path, 'tractor-{brickname}.fits'.format(brickname=bricks[0]))
tractor_header = fitsio.read_header(tractor_filename, ext=1)
print tractor_header


XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / 8-bit bytes
NAXIS   =                    2 / 2-dimensional binary table
NAXIS1  =                 1326 / width of table in bytes
NAXIS2  =                 3018 / number of rows in table
PCOUNT  =                    0 / size of special data area
GCOUNT  =                    1 / one data group (required keyword)
TFIELDS =                   69 / number of fields in each row
TTYPE1  = 'brickid '           / label for field   1
TFORM1  = 'J       '           / data format of field: 4-byte INTEGER
TTYPE2  = 'brickname'          / label for field   2
TFORM2  = '8A      '           / data format of field: ASCII Character
TTYPE3  = 'objid   '           / label for field   3
TFORM3  = 'J       '           / data format of field: 4-byte INTEGER
TTYPE4  = 'brick_primary'      / label for field   4
TFORM4  = 'L       '           / data format of field: 1-byte LOGICAL
TTYPE5  = 'blob    '           / label for 